In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sbrn
import numpy as np
import re
import datetime
import impute
import pump_modeling_2 as pm
from sklearn.cross_validation import train_test_split

%matplotlib inline

In [2]:
pm = pm.PumpModel(csv_train_X='/Users/ash/ds/projects/data/water_pump/training.csv', 
                  csv_train_y='/Users/ash/ds/projects/data/water_pump/training_labels.csv',
                  csv_test_X='/Users/ash/ds/projects/data/water_pump/test.csv'
                 )

PumpModel obj. initialzed. data read into df. 
PumpModel.run_batch() to see quick analysis results.


In [ ]:
pm.df[['id']]

In [ ]:
pm.df.apply(lambda col: col.isnull().sum(), axis=0)]


In [ ]:
pm.df.fillna('UNKNOWN', axis=0, inplace=True) #replace all categorical missing values with 'UNKNOWN'


In [ ]:
pm.df.apply(lambda col: col.isnull().sum(), axis=0)



In [ ]:
pm.df.columns


In [ ]:
Xcols = [u'gps_height', u'longitude', u'latitude',
       u'basin', u'region_code', u'population', u'subvillage', u'ward',u'lga',
       u'scheme_management', u'construction_year', u'extraction_type_group', u'payment', u'water_quality',
       u'quantity', u'source', u'waterpoint_type']


In [ ]:
#split test train, impute train, train, then test
Xtrain, Xtest, ytrain, ytest = train_test_split(pm.df.loc[:,Xcols],pm.df.status,train_size=0.7,random_state=666)


In [ ]:
(Xtrain.shape, Xtest.shape)


In [ ]:
train, imap = impute.imputeTrain(Xtrain) #impute missing numeric predictors and train reference map


In [ ]:
train.shape

In [ ]:
#remove too many categories for first run
trainCols = [u'gps_height', u'longitude', u'latitude',
       u'basin', u'region_code', u'population',
       u'scheme_management', u'construction_year', u'extraction_type_group', u'payment', u'water_quality',
       u'quantity', u'source', u'waterpoint_type']

train = train.loc[:,trainCols]

train.shape

In [ ]:
#get dummies!

def getDumDums(dat):
    """
    Takes in a dataframe and returns dataframe with all the 'object' columns into dummies without prejudice.
    Might take a while depending on # of categoricals
    """
    catCols = [col for col in dat.columns if dat[col].dtype=='object']
    
    return pd.get_dummies(dat,prefix_sep='__', columns=catCols)




In [ ]:
train1 = getDumDums(train)

In [ ]:
train1.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier




In [ ]:
rm=RandomForestClassifier(n_estimators=300, criterion='entropy', verbose=1, oob_score=True)
rm.fit(train1,ytrain)


In [ ]:
#preprocess Xtest ready
Xtest.shape

test1 = impute.fillTest(Xtest,imap)
test1.shape

In [ ]:
test1 = test1.loc[:,trainCols]

test1.shape

In [ ]:
test1 = getDumDums(test1)

In [ ]:
str(set(train1.columns)-set(test1.columns))

In [ ]:
test1[str((train1.columns)-set(test1.columns))]=0

In [ ]:
test1.shape

In [ ]:
rm.score(test1, ytest)



In [ ]:
rm.predict(test1)

In [ ]:
ytest.head()

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(ytest, rm.predict(test1),labels=[0,1,2])

In [ ]:
rm.score(test1,ytest)

In [ ]:
from sklearn.cross_validation import KFold

In [ ]:
train1 = sm.add_constant(train1, prepend=True, has_constant='skip')
test1 = sm.add_constant(test1, prepend=True, has_constant='skip')
lm = sm.MNLogit(ytrain, train1)

In [ ]:
lm.fit()
lm.score(test1)

In [ ]:
train1 = sm.add_constant(train1, prepend=True, has_constant='skip')
test1 = sm.add_constant(test1, prepend=True, has_constant='skip')

In [ ]:
train1.shape, test1.shape

In [ ]:
set(test1.columns)-set(train1.columns)

In [ ]:
test1.drop(['scheme_management__None'], inplace=True, axis=1)

In [ ]:
lm.predict(test1)